In [143]:
%pylab inline
from scipy import signal
import math
import numpy as np
pylab.rcParams['figure.figsize'] = (16, 6)
pylab.rcParams["font.size"] = "14"
import IPython.display as ipd
from ipywidgets import interact, fixed
import ipywidgets as widgets

# settings
SRATE = 22050

# note names to freq for widgets
notes = 'C-,C#,D-,D#-,E-,F-,F#,G-,G#,A-,A#,B-,C-'.split(',')
notes = [n + str(o) for o in range(1,5) for n in notes]
freqs = 55. * 2**(np.arange(3, 3 + len(notes)) / 12.)
notes = list(zip(notes, freqs))
#print(notes)

Populating the interactive namespace from numpy and matplotlib


In [144]:
def display_wave(a, t, srat=SRATE):
    figure(1)
    # waveform
    plot(t, a)
    xlabel('Time (s)')
    grid(True)
    margins(0.0)

    plt.show()

    # audio player
    ipd.display(ipd.Audio(a, rate=srat))

In [145]:
def drones(frq, ticks, bpm, tpb, voices=2, srat=SRATE):
    # TODO: add cycle_inc parameter
    # TODO: let user choose waveform (sin?, saw, sqr)
    # TODO: make stereo and spread voices
    # TODO: other combine modes (AM?)
    
    # from ticks and samplerate we calculate the number of samples for desired resync.   
    # ticks per second
    tps = (bpm * tpb) / 60.0
    # samples per tick
    spt = srat / tps
    dur = ticks * spt
    
    print("Duration samples: %f" %  dur)
    
    # from the tone frequency we can check what the phase would be at the point of resync.
    # frq = 110 hz
    # samples_per_cycle = srat / frq
    spc = srat / frq
    # we round this to end with phase=0 at the end of the loop
    # TODO: do a table with the error for the whole tone range and various bpms
    num_cycles = round(dur / spc)
    print("Num cycles (rounded): %f" %  num_cycles)
    frqs=[]
    for v in range(voices):
        frqs.append(srat / (dur / num_cycles))
        num_cycles +=1
    
    print("Frq error: ", abs(frq - frqs[0]))
    print("Frqs: ", *frqs)

    twopi = 2.0 * np.pi
    t = np.linspace(0, dur/srat, int(dur))
    a = np.zeros(len(t))
    for v in range(voices):
        #a += np.sin(t * twopi * frqs[v])
        a += signal.sawtooth(t * twopi * frqs[v])
    
    display_wave(a, t)

In [148]:
interact(drones, 
         frq=notes, 
         ticks=widgets.IntSlider(min=1, max=256, step=1, value=16, continuous_update=False), 
         bpm=widgets.IntSlider(min=60, max=300, step=1, value=125, continuous_update=False), 
         tpb=widgets.IntSlider(min=1, max=16, step=1, value=4, continuous_update=False),
         voices=widgets.IntSlider(min=1, max=10, step=1, value=3),
         srat=fixed(SRATE))

<function __main__.drones>